In [1]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install selenium-stealth

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1080")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('ignore-certificate-errors')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.page_load_strategy = 'eager'

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.auto import tqdm
import numpy as np

In [3]:
ser = Service("C:\chromedriver\chromedriver.exe")
driver = webdriver.Chrome(service=ser, options=options)

stealth(
    driver,
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
    languages = ["en-US", "en"],
    vendor = "Google Inc.",
    platform = "Win32",
    webgl_vendor = "Intel Inc.",
    renderer = "Intel Iris OpenGL Engine",
    fix_hairline = False,
    run_on_insecure_origins = False,
)

In [4]:
#Import Dataset
df_src = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                   '1U0n5tunbsXHvBRi_r5d3kuSXoJy_tutDxhOe2TiOwqM' +
                   '/export?gid=1390430554&format=csv')

df_src['url'][112:113]

112    http://103.131.61.7/eproc4
Name: url, dtype: object

In [5]:
# function halaman perubahan
# kalo di pengumuman ada parameter tahun, cuma nanti kita selidiki
def get_data_perubahan(link, link_tahapan, lpse_link):
    driver.get(link)
    time.sleep(0.5)
    data = BeautifulSoup(driver.page_source, 'lxml')
    for a in data.find_all('tr')[2:]:
        kode_perubahan = re.findall('\d{4,14}', link)[0]
        list_kode_perubahan.append(kode_perubahan)
        
        kode_tender_perubahan = re.findall('\d{4,14}', link_tahapan)[0]
        list_kode_tender_perubahan.append(kode_tender_perubahan)
        
        list_link_perubahan.append(link)

        list_url_lpse_perubahan.append(lpse_link)

        tanggal = a.select_one('td:nth-of-type(2)').text
        list_tanggal_perubahan.append(tanggal)

        mulai = a.select_one('td:nth-of-type(3)').text
        list_mulai_perubahan.append(mulai)

        sampai = a.select_one('td:nth-of-type(4)').text
        list_sampai_perubahan.append(sampai)

        keterangan = a.select_one('td:nth-of-type(5)').text
        list_keterangan_perubahan.append(keterangan)


In [6]:
def get_data_halaman_tender():
    data_tender = BeautifulSoup(driver.page_source, 'lxml')
    # looping to get main data from one url
    for area in tqdm(data_tender.find_all('tr', {'class': ['odd','even']})):
        print('Loop TR Tender')
        kode_paket = area.find('td', class_="sorting_1").text

        if kode_paket in kode_paket_list:
          continue
        else:
          kode_paket_list.append(kode_paket)
          get_data_halaman_tahapan(kode_paket)

def get_data_halaman_tahapan(kode_paket):
    link_tahapan = "{}".format(link_lpse + "/"+ kode_paket +"/jadwal")
    driver.get(link_tahapan)
    # time.sleep(0.5)
    data_tahapan = BeautifulSoup(driver.page_source, 'lxml')
    content_tahapan = data_tahapan.find('div', class_="content")
    for a in data_tahapan.find_all('tr')[1:]:
        list_url_lpse_tahapan.append(link_lpse)      

        list_kode_tender_tahapan.append(kode_paket)

        tahap =  a.select_one('td:nth-of-type(2)').text.replace("\n", "").strip()
        list_tahap_tahapan.append(tahap)

        mulai = a.select_one('td:nth-of-type(3)').text
        list_mulai_tahapan.append(mulai)

        sampai = a.select_one('td:nth-of-type(4)').text
        list_sampai_tahapan.append(sampai)

        perubahan =  a.select_one('td:nth-of-type(5)').text.strip()
        list_perubahan_tahapan.append(perubahan)
        if "perubahan" in perubahan:
            tautan_kode = a.select_one('a[href]').get('href')
            link = link_lpse.replace('/eproc4/lelang', '')
            url_perubahan = link + tautan_kode
            get_data_perubahan(url_perubahan, link_tahapan, link_lpse)

            kode_perubahan_tahapan = re.findall('\d{4,14}', url_perubahan)[0]
            list_kode_perubahan_tahapan.append(kode_perubahan_tahapan)
            list_link_perubahan_tahapan.append(url_perubahan)
            
        if "Tidak Ada" in perubahan:
            list_link_perubahan_tahapan.append("")
            list_kode_perubahan_tahapan.append("")
    
    if content_tahapan == None:
      print("link error tahapan = ", link_tahapan)
 
      
def get_data_filterjumlah(jumlah):
    # Pilih jumlah data per halaman yg ingin ditampilkan
    paginasi = False
    driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
    if jumlah == 'Semua':
        jumlah_data = driver.find_element(By.XPATH,"//div[@class='dataTables_info']").text.split(' ')[-2].replace(',','')
        if int(jumlah_data) < 1000:
          print('datanya < 1000')
        elif int(jumlah_data) > 5000:
          print('datanya > 5000')
        else:
          print('di antara')
        
        data_tmp = BeautifulSoup(driver.page_source, 'lxml')
        tr_tender = data_tmp.select('table[id="tbllelang"] tbody tr')
        if len(tr_tender) == 25:
          paginasi = True
          jumlah = 100
    else:
        paginasi = True
        # time.sleep(2)
    # Jumlah paginasi halaman
    page_list = driver.find_elements(By.XPATH, "//ul[@class='pagination']//li")
    last = int(page_list[-3].text)
    for i in tqdm (range(1,last+1)):
        print('Loop Paginasi')
        get_data_halaman_tender()
        if paginasi:
            driver.get(link_lpse)
            # time.sleep(1)
            driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
            # time.sleep(1)
            for j in range(i):
                driver.execute_script("arguments[0].click()", driver.find_element(By.XPATH, "//li[@id='tbllelang_next']"))
                # time.sleep(2)
        
        
def get_data_filtertahun(tahun, jumlah):
    # get website
    driver.get(link_lpse)
    # waiting to website load succesfully
    # time.sleep(1)
    # Filter tahun sesuai parameter
    Select(driver.find_element(By.XPATH, "//select[@name='tahun']")).select_by_value(tahun)
    # time.sleep(1)
    get_data_filterjumlah(jumlah)

In [7]:
#variabel data tender
kode_paket_list = []

#variabel data tahapan
list_url_lpse_tahapan = []
list_link_perubahan_tahapan = []
list_kode_tender_tahapan = []
list_kode_perubahan_tahapan = []
list_tahap_tahapan = []
list_mulai_tahapan = []
list_sampai_tahapan = []
list_perubahan_tahapan = []

#variabel data perubahan
list_url_lpse_perubahan = []
list_link_perubahan = []
list_kode_tender_perubahan = []
list_kode_perubahan = []
list_tanggal_perubahan = []
list_mulai_perubahan = []
list_sampai_perubahan = []
list_keterangan_perubahan = []

In [8]:
# set Url based on index in dataset
i= 199
end= 200

# Looping to put data in variables
for i in tqdm(range(i, end)):
        print('Loop LPSE')
        link_lpse = "{}".format(df_src['url'].iloc[i]+"/lelang")
        driver.get(link_lpse)
        # time.sleep(2)
        tampilan = driver.find_elements(By.XPATH, "//select[@name='tbllelang_length']//option")
        jumlah_total = driver.find_element(By.XPATH,"//div[@class='dataTables_info']").text.split(' ')[-2].replace(',','')
        data_home_default = BeautifulSoup(driver.page_source, 'lxml')
        years = [e.text for e in data_home_default.select('select[name="tahun"] option')[1:-1]]
        years.sort(reverse=True)
        if years == [] or len(tampilan) < 4 :
            get_data_filterjumlah('100')
        else:
            for year in tqdm(years):
                print('Loop Tahun')
                get_data_filtertahun(year, 'Semua')
              
        if len(np.unique(list_kode_tender_tahapan)) != int(jumlah_total):
          driver.get(link_lpse)
          time.sleep(2)
          driver.find_element(By.NAME,"tbllelang_length").send_keys('Semua')
          if int(jumlah_total) < 5000:
            time.sleep(2)
          elif int(jumlah_total) > 10000:
            time.sleep(4)
          else:
            time.sleep(2)
          get_data_halaman_tender()

                    

  0%|          | 0/1 [00:00<?, ?it/s]

Loop LPSE


  0%|          | 0/10 [00:00<?, ?it/s]

Loop Tahun
datanya < 1000


  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop Paginasi


  0%|          | 0/25 [00:00<?, ?it/s]

Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender
Loop TR Tender


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@id='tbllelang_next']"}
  (Session info: headless chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C91ED3+2236115]
	Ordinal0 [0x00C292F1+1807089]
	Ordinal0 [0x00B366FD+812797]
	Ordinal0 [0x00B655DF+1005023]
	Ordinal0 [0x00B657CB+1005515]
	Ordinal0 [0x00B97632+1209906]
	Ordinal0 [0x00B81AD4+1120980]
	Ordinal0 [0x00B959E2+1202658]
	Ordinal0 [0x00B818A6+1120422]
	Ordinal0 [0x00B5A73D+960317]
	Ordinal0 [0x00B5B71F+964383]
	GetHandleVerifier [0x00F3E7E2+2743074]
	GetHandleVerifier [0x00F308D4+2685972]
	GetHandleVerifier [0x00D22BAA+532202]
	GetHandleVerifier [0x00D21990+527568]
	Ordinal0 [0x00C3080C+1837068]
	Ordinal0 [0x00C34CD8+1854680]
	Ordinal0 [0x00C34DC5+1854917]
	Ordinal0 [0x00C3ED64+1895780]
	BaseThreadInitThunk [0x76ADFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77AF7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77AF7B8E+238]


In [9]:
 dictionary1 = {
    "url_lpse": list_url_lpse_tahapan,
    "url_perubahan": list_link_perubahan_tahapan,
    "kode_tender": list_kode_tender_tahapan,
    "kode_perubahan": list_kode_perubahan_tahapan,
    "tahap": list_tahap_tahapan,
    "tgl_mulai": list_mulai_tahapan,
    "tgl_akhir": list_sampai_tahapan,
    "perubahan": list_perubahan_tahapan
}

df_tahapan = pd.DataFrame.from_dict(dictionary1, orient='index')
df_tahapan = df_tahapan.T
df_tahapan

,url_lpse,url_perubahan,kode_tender,kode_perubahan,tahap,tgl_mulai,tgl_akhir,perubahan
0,http://lpse.kutaitimurkab.go.id/eproc4/lelang,,79614364,,Pengumuman Pascakualifikasi,19 Juli 2022 16:00,25 Juli 2022 10:00,Tidak Ada
1,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61482364,Download Dokumen Pemilihan,19 Juli 2022 16:00,26 Juli 2022 10:00,1 kali perubahan
2,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61483364,Pemberian Penjelasan,21 Juli 2022 10:00,22 Juli 2022 13:30,1 kali perubahan
3,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61484364,Upload Dokumen Penawaran,22 Juli 2022 15:00,26 Juli 2022 10:00,1 kali perubahan
4,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61485364,Pembukaan Dokumen Penawaran,26 Juli 2022 10:01,26 Juli 2022 23:59,1 kali perubahan
...,...,...,...,...,...,...,...,...
60970,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17085364,Penetapan Pemenang,25 Oktober 2013 09:00,25 Oktober 2013 15:00,2 kali perubahan
60971,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17086364,Pengumuman Pemenang,25 Oktober 2013 16:00,25 Oktober 2013 23:59,2 kali perubahan
60972,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17087364,Masa Sanggah,28 Oktober 2013 08:00,30 Oktober 2013 16:00,1 kali perubahan
60973,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17088364,Surat Penunjukan Penyedia Barang/Jasa,31 Oktober 2013 08:00,31 Oktober 2013 16:00,1 kali perubahan


# Alert! Save Dataframe

In [10]:
 with pd.ExcelWriter(f"tahapan_{i}.xlsx", options={'strings_to_urls': False}) as writer:
        df_tahapan.to_excel(writer, index=False)

C:\Users\Asus\AppData\Local\Temp/ipykernel_3560/1674282203.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter(f"tahapan_{i}.xlsx", options={'strings_to_urls': False}) as writer:


JUMLAH KODE PAKET

In [11]:
len(np.unique(list_kode_tender_tahapan))

3900

In [13]:
dictionary2 = {
    "url_lpse": list_url_lpse_perubahan,
    "url_perubahan": list_link_perubahan,
    "kode_tender": list_kode_tender_perubahan,
    "kode_perubahan": list_kode_perubahan,
    "tanggal_edit": list_tanggal_perubahan,
    "tgl_mulai": list_mulai_perubahan,
    "tgl_akhir": list_sampai_perubahan,
    "keterangan": list_keterangan_perubahan
}

df_perubahan = pd.DataFrame.from_dict(dictionary2, orient='index')
df_perubahan = df_perubahan.T
df_perubahan

,url_lpse,url_perubahan,kode_tender,kode_perubahan,tanggal_edit,tgl_mulai,tgl_akhir,keterangan
0,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61482364,22 Juli 2022 13:29,19 Juli 2022 16:00,25 Juli 2022 10:00,Memperpanjang masa Download Dokumen Pemilihan
1,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61483364,22 Juli 2022 13:29,21 Juli 2022 10:00,21 Juli 2022 11:00,Memperpanjang masa Download Dokumen Pemilihan
2,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61484364,22 Juli 2022 13:29,21 Juli 2022 14:00,25 Juli 2022 10:00,Memperpanjang masa Download Dokumen Pemilihan
3,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61485364,22 Juli 2022 13:29,25 Juli 2022 10:01,25 Juli 2022 23:59,Memperpanjang masa Download Dokumen Pemilihan
4,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,79614364,61486364,2 Agustus 2022 13:00,26 Juli 2022 11:00,2 Agustus 2022 14:00,"memperpanjang masa Evaluasi Administrasi, Kual..."
...,...,...,...,...,...,...,...,...
50745,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17086364,25 Oktober 2013 09:30,25 Oktober 2013 11:00,25 Oktober 2013 23:59,Penyesuain Waktu pada Tahapan Penetapan Pemenang
50746,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17086364,9 Oktober 2013 12:25,30 September 2013 11:00,30 September 2013 23:59,
50747,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17087364,9 Oktober 2013 12:25,1 Oktober 2013 08:00,3 Oktober 2013 16:00,
50748,http://lpse.kutaitimurkab.go.id/eproc4/lelang,http://lpse.kutaitimurkab.go.id/eproc4/jadwal/...,1148364,17088364,9 Oktober 2013 12:25,4 Oktober 2013 08:00,4 Oktober 2013 16:00,


# Alert! Save Dataframe

In [14]:
 with pd.ExcelWriter(f"perubahan_{i}.xlsx", options={'strings_to_urls': False}) as writer:
        df_tahapan.to_excel(writer, index=False)

C:\Users\Asus\AppData\Local\Temp/ipykernel_3560/342319117.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter(f"perubahan_{i}.xlsx", options={'strings_to_urls': False}) as writer:
